# Politician Activity on Facebook by Political Affiliation

The parameters in the cell below can be adjusted to explore other political affiliations and time frames.

### How to explore other political affiliation?
The ***affiliation*** parameter can be use to aggregate politicians by their political affiliations. The column `affiliation` in this [this other notebook](../politicians.ipynb?autorun=true) show the politicians that belong each political affiliation.

***Alternatively***, you can direcly use the [politicians API](http://mediamonitoring.gesis.org/api/politicians/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

## A. Set Up parameters

In [ ]:
# Parameters: 
affiliation = 'AfD'
from_date = '2017-09-01'
to_date = '2018-12-31'
aggregation = 'week'

## B. Using the SMM Politician API

In [ ]:
library("httr")
library("jsonlite")
library('dplyr', warn.conflicts = FALSE)

# prepare urls
base <- "http://mediamonitoring.gesis.org/api/politicians/"
url_all <- paste(base,"all/", sep="")
url_posts <- paste(base, "facebook/posts_by/politicians/", sep="")
url_comments <- paste(base, "facebook/comments_by/politicians/", sep="")

# prepare parameters
params = list(
    from_date=from_date,
    to_date=to_date,
    aggregate_by=aggregation
)

# use the api to get the politicians and filter the parties
orgs <- as.data.frame(fromJSON(content(GET(url_all), "text", encoding="UTF-8"), flatten = TRUE))
party_df <- orgs[orgs$affiliation == affiliation & !is.null(orgs$fb_ids) ,]


# query the Social Media Monitoring API for posts and comments
posts <- data.frame()
comments <- data.frame()
for (politician_id in party_df$politician_id) {
    json_posts <- fromJSON(content(GET(paste(url_posts, politician_id, "/?", sep=""),query=params), "text", encoding="UTF-8"), flatten = TRUE)
    json_comments <- fromJSON(content(GET(paste(url_comments, politician_id, "/?", sep=""), query=params), "text", encoding="UTF-8"), flatten = TRUE)

    # concatenate
    if (length(json_posts$values) != 0) {
        posts <- rbind(posts, as.data.frame(json_posts))
    }    
    if (length(json_comments$values) != 0) {
        comments <- rbind(comments, as.data.frame(json_comments))
    }
}

# group by day, week, or month, and then merge
posts <- summarise(group_by(posts, labels), posts = sum(values), shares = sum(shares), replies = sum(replies), reactions = sum(reactions), likes = sum(likes), response_type = aggregation)
comments <- summarise(group_by(comments, labels), comments = sum(values), replies = sum(replies), likes = sum(likes), response_type = aggregation)
merged <- merge(posts, comments, by='labels')

## C. Plotting
### C.1 Plot Facebook Post Activity

In [ ]:
library("ggplot2")
options(repr.plot.width=8, repr.plot.height=4)

#plotting facebook posts
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = posts, color="Posts", group=response_type.x)) +
  geom_line(aes(y = replies.x, color="Replies", group=response_type.x)) +
  geom_line(aes(y = shares, color="Shares", group=response_type.x)) +
  geom_line(aes(y = reactions, color="Reactions", group=response_type.x)) +
  geom_line(aes(y = likes.x, color="Likes", group=response_type.x)) +
  labs(title = "Facebook (Posts Activity)", y = "Posts") +
  theme(axis.text.x = element_text(angle = 60, hjust = 0.5, vjust = 0.5),
        axis.title.x = element_blank(), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 weeks") 

### C.2 Plot Facebook Comment Activity

In [ ]:
#plotting facebook comments
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = comments, color="Comments", group=response_type.y)) +
  geom_line(aes(y = replies.y, color="Replies", group=response_type.y)) +
  geom_line(aes(y = likes.y, color="Likes", group=response_type.x)) +
  labs(title = "Facebook (Comments Activity)", y = "Comments") +
  theme(axis.text.x = element_text(angle = 60, hjust = 0.5, vjust = 0.5),
        axis.title.x = element_blank(), legend.title = element_blank()) +
scale_x_date(date_breaks = "1 weeks") 